In [1]:
from dataclasses import dataclass, field
from typing import Callable, Optional, Any
import pandas as pd
import numpy as np

from reportlab.platypus import Table, TableStyle
from reportlab.lib import colors


# -------------------------------
# Column formatting schema
# -------------------------------
@dataclass
class ColumnFormat:
    digits: int = 2
    comma: bool = False
    formatter: Optional[Callable[[Any], str]] = None  # e.g., lambda x: f"${x:,.2f}"
    colormap: Optional[Callable[[float, float, float], colors.Color]] = None
    # colormap takes (val, vmin, vmax) and returns a ReportLab Color


@dataclass
class ColumnMeta:
    name: str
    label: Optional[str] = None
    fmt: ColumnFormat = field(default_factory=ColumnFormat)


# -------------------------------
# Helper functions
# -------------------------------
def default_number_formatter(val, digits=2, comma=False):
    if pd.isna(val):
        return ""
    fmt = f"{{:,.{digits}f}}" if comma else f"{{:.{digits}f}}"
    return fmt.format(val)


def build_table_from_df(df: pd.DataFrame, schema: list[ColumnMeta]) -> Table:
    """Turn DataFrame + schema into a styled ReportLab Table."""

    # --- Build data matrix (header + rows)
    headers = [col.label or col.name for col in schema]
    table_data = [headers]

    # Precompute vmin/vmax for each col needing a colormap
    vmin_vmax = {}
    for col in schema:
        if col.fmt.colormap:
            series = pd.to_numeric(df[col.name], errors="coerce")
            vmin_vmax[col.name] = (series.min(), series.max())

    for _, row in df.iterrows():
        row_data = []
        for col in schema:
            val = row[col.name]

            # custom formatter > generic number formatter > str
            if col.fmt.formatter:
                display_val = col.fmt.formatter(val)
            elif isinstance(val, (int, float, np.number)):
                display_val = default_number_formatter(
                    val, digits=col.fmt.digits, comma=col.fmt.comma
                )
            else:
                display_val = str(val)
            row_data.append(display_val)
        table_data.append(row_data)

    # --- Build ReportLab table
    tbl = Table(table_data, repeatRows=1)
    style = TableStyle([
        ("GRID", (0,0), (-1,-1), 0.5, colors.black),
        ("BACKGROUND", (0,0), (-1,0), colors.lightgrey),  # header
        ("ALIGN", (0,0), (-1,-1), "CENTER"),
    ])

    # --- Apply colormap cell backgrounds
    for row_idx, (_, row) in enumerate(df.iterrows(), start=1):
        for col_idx, col in enumerate(schema):
            if col.fmt.colormap:
                vmin, vmax = vmin_vmax[col.name]
                val = row[col.name]
                if pd.notna(val):
                    bgcolor = col.fmt.colormap(val, vmin, vmax)
                    style.add("BACKGROUND", (col_idx, row_idx), (col_idx, row_idx), bgcolor)

    tbl.setStyle(style)
    return tbl


In [2]:
from reportlab.lib import colors

def custom_diverging_colormap(low_color=(0, 1, 0), mid_color=(1, 1, 1), high_color=(1, 0, 0)):
    """
    Return a function that maps values to colors.
    
    low_color, mid_color, high_color are RGB tuples (0-1 scale).
    If mid_color=None, it will just interpolate low → high.
    """

    def _map(val, vmin, vmax, vmid=None):
        if vmax == vmin:  # avoid division by zero
            return colors.Color(*mid_color if mid_color else low_color)

        # default mid = midpoint
        if vmid is None:
            vmid = (vmax + vmin) / 2.0

        if val <= vmid:
            # interpolate low → mid
            t = (val - vmin) / (vmid - vmin) if vmid > vmin else 0
            r = low_color[0] + t * (mid_color[0] - low_color[0])
            g = low_color[1] + t * (mid_color[1] - low_color[1])
            b = low_color[2] + t * (mid_color[2] - low_color[2])
        else:
            # interpolate mid → high
            t = (val - vmid) / (vmax - vmid) if vmax > vmid else 0
            r = mid_color[0] + t * (high_color[0] - mid_color[0])
            g = mid_color[1] + t * (high_color[1] - mid_color[1])
            b = mid_color[2] + t * (high_color[2] - mid_color[2])

        return colors.Color(r, g, b)

    return _map


In [ ]:
import matplotlib
from reportlab.platypus import SimpleDocTemplate, Spacer, Paragraph
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet

def hex_to_rgb01(hex_str: str):
    hex_str = hex_str.lstrip("#")
    return tuple(int(hex_str[i:i+2], 16) / 255.0 for i in (0, 2, 4))

hex_to_rgb01("#FF0000")  # -> (1.0, 0.0, 0.0)


def mpl_colormap(cmap_name="RdYlGn"):
    cmap = matplotlib.colormaps[cmap_name]   # ✅ modern API
    def _map(val, vmin, vmax):
        norm = (val - vmin) / (vmax - vmin) if vmax > vmin else 0.5
        r, g, b, _ = cmap(norm)
        return colors.Color(r, g, b)
    return _map

arb_cmap = custom_diverging_colormap(
    low_color=(0, 1, 0),  # green
    mid_color=(1, 1, 1),    # white
    high_color=(1, 0, 0)  # red
)

# Define schema
schema = [
    ColumnMeta("Name", label="Name"),
    ColumnMeta("Score",
               label="Score",
               fmt=ColumnFormat(digits=0, comma=False, colormap=arb_cmap)),
    ColumnMeta("Balance",
               label="Balance ($)",
               fmt=ColumnFormat(digits=2, comma=True)),
]

# Example DataFrame
df = pd.DataFrame({
    "Name": ["Alice", "Bob", "Charlie", "Diana", "Eve"],
    "Score": [23, 45, 12, 78, 56],
    "Balance": [12345.6, 98765.43, 500.25, 3200.7, 65000.0],
})

# Build PDF
doc = SimpleDocTemplate("styled_table.pdf", pagesize=letter)
story = []

story.append(Paragraph("Demo: Styled Table", getSampleStyleSheet()["Heading1"]))
story.append(Spacer(1, 12))

tbl = build_table_from_df(df, schema)
story.append(tbl)

doc.build(story)
